In [1]:
import pandas as pd
import subprocess
import datetime as dt

In [2]:
image_file = pd.read_csv('new_image_bbox.csv')

In [3]:
## Retrieve tar files from the cyverse data store for specified season and day ex: ('season_10_yr_2020', '2020-03-03')
class Thermal:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/level_1/{season}/EnvironmentLogger/{date}_clean.tar.gz'
        subprocess.call(command, shell = True)
        command = f'tar -xvf {date}_clean.tar.gz'
        subprocess.call(command, shell = True)

## Retrieve csv data and organize/clean up
    def retrieve_data(season, date):
        tar_data = Thermal.get_tar(season, date)
        data = pd.read_csv(f'./{date}_clean.csv')
        data['Time'] = pd.to_datetime(data['Time'])
        #data['date'] = data.Time.dt.date
        #data['time'] = data.Time.dt.time
        data_clean = data[['Time', 'Sun Direction', 'Temperature', 
                                      'Photosynthetically active radiation', 'Wind velocity']]
        #index_data = data_clean.set_index('time')
        return data_clean

In [4]:
## Searches an already downloaded file of the year's AZmet data
## Has the capability of retrieving weather data for the specific date and hour

## Labels the columns with their appropriate header
class AZmet:
    def __init__ (self, filename):
        file = pd.read_csv(filename, names = ["Year", "Day", "Hour", 
                                            "Air Temperature", "Relative Humidity", 
                                            "VPD", "Solar Radiation", "Precipitation", 
                                            "4 inch Soil T", "12 inch Soil T", 
                                            "Avg Wind Speed", "Wind Vector Magnitude", 
                                            "Wind Vector Direction", "Wind Direction STDEV", 
                                            "Max Wind Speed", "Reference Evapotranspiration", 
                                            "Actual Vapor Pressure", "Dewpoint"])
        file_df = pd.DataFrame(file)
        file_df['combined'] = file_df["Year"]*1000 + file_df["Day"]
        file_df["date"] = pd.to_datetime(file_df["combined"], format = "%Y%j")
        data = file_df.set_index('date')
        del data['combined']
        self.__data__ = data

## Selects the data for the specific Date
    def specific_time(self, date, hour):
        data = self.__data__
        specific_time = data[data.index == date]
        return specific_time

## Selects the data for the specific hour
    def specific_hour(self, date, hour):
        date_data = AZmet.specific_time(self, date, hour)
        hour = date_data[date_data["Hour"] == hour]
        return hour

## Insert AZmet file for that year here
filename = AZmet('/Users/sebastiancalleja/Desktop/AZmet_Data/2020_azmet_data.txt')

In [5]:
## Environmental logger data for specified date
EL = Thermal.retrieve_data('season_10_yr_2020', '2020-03-03')

In [6]:
## azmet data for specified date
AZmet = pd.read_csv('/Users/sebastiancalleja/Desktop/AZmet_Data/azmet_2020_03-03.csv')

In [7]:
## Converts azmet date to datetime
AZmet['date'] = pd.to_datetime(AZmet['date'])

In [8]:
image_file['Date and Time'] = pd.to_datetime(image_file['Date and Time'])

In [9]:
## Gathers environmental logger temp data for each plot based on 'nearest' HH:MM time
Env_temp = []
def env_temp(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = EL['Time'].sub(time).abs().idxmin()
        EL_result = EL['Temperature'].iloc[result_index]
        Env_temp.append(EL_result)
    return (Env_temp)

In [10]:
## Gathers environmental logger wind data for each plot based on 'nearest' HH:MM time
Env_wind = []
def env_wind(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = EL['Time'].sub(time).abs().idxmin()
        EL_result = EL['Wind velocity'].iloc[result_index]
        Env_wind.append(EL_result)
    return (Env_wind)

In [11]:
image_file['Env Temperature'] = env_temp(image_file)

In [12]:
image_file['Env Wind Velocity'] = env_wind(image_file)

In [9]:
## Creates a dictionary of all of the AZmet data of interest
AZmet_dict = {}
def azmet_dict(image_file):
    cnt = 0
    for i, row in image_file.iterrows():
        #global cnt
        cnt += 1
        time = row['Date and Time']
        result_index = AZmet['date'].sub(time).abs().idxmin()
        AZmet_temp = AZmet['Air Temperature'].iloc[result_index]
        AZmet_wind = AZmet['Avg Wind Speed'].iloc[result_index]
        AZmet_vpd = AZmet['VPD'].iloc[result_index]
        AZmet_solar = AZmet['Solar Radiation'].iloc[result_index]
        AZmet_rh = AZmet['Relative Humidity'].iloc[result_index]
        Env_temp = EL['Temperature'].iloc[result_index]
        Env_wind = EL['Wind velocity'].iloc[result_index]
        AZmet_dict[cnt] = {'azmet_atm_temp': AZmet_temp, 'azmet_wind_velocity': AZmet_wind, 'azmet_VPD': AZmet_vpd, 'azmet_solar_radiation':
                          AZmet_solar, 'relative_humidity': AZmet_rh, 'env_temp': Env_temp, 'env_wind': Env_wind}
    return pd.DataFrame.from_dict(AZmet_dict)

In [10]:
environmental_df = azmet_dict(image_file)

In [11]:
environmental_df = environmental_df.transpose()

In [13]:
## Adds the AZmet data from the dictionary above to the master csv
image_file['azmet_atm_temp'] = environmental_df['azmet_atm_temp']
image_file['azmet_wind_velocity'] = environmental_df['azmet_wind_velocity']
image_file['azmet_VPD'] = environmental_df['azmet_VPD']
image_file['azmet_solar_radiation'] = environmental_df['azmet_solar_radiation']
image_file['relative_humidity'] = environmental_df['relative_humidity']
image_file['env_temp'] = environmental_df['env_temp']
image_file['env_wind'] = environmental_df['env_wind']

In [15]:
del image_file['Unnamed: 0']

In [16]:
full_image_file = image_file

In [17]:
full_image_file.to_csv('2020-03-03_all_atm_temp.csv')

In [19]:
## Modifies the csv to be used in conjunction with the plant detection csv
full_image_file_mod = full_image_file[['Date and Time', 'Image Name', 'azmet_atm_temp', 'azmet_wind_velocity', 'azmet_VPD', 
                                      'azmet_solar_radiation', 'relative_humidity', 'env_temp','env_wind']]

In [20]:
full_image_file_mod = full_image_file_mod.set_index('Image Name')

In [21]:
full_image_file_mod.to_csv('full_image_file_mod.csv')